In [1]:
import os

In [2]:
%pwd

'/Users/miguelhermar/Desktop/Projects/MLflow-Project/research'

In [3]:
os.chdir("../") # go back to parent folder

In [4]:
%pwd

'/Users/miguelhermar/Desktop/Projects/MLflow-Project'

# Update entity folder

In [5]:
from dataclasses import dataclass # define the variable instead of using "self" for classes
from pathlib import Path


@dataclass(frozen=True) # not a python class, but a data class
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

## Update the configuration manager in src config folder

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir, # creating root directory (data ingestion directory)
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

## Update components (how the data will be ingested)

In [8]:
import os
import urllib.request as request # to download data from the url
import zipfile # to unzip the data
from mlProject import logger
from mlProject.utils.common import get_size # see file size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

 
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

## Update pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-29 20:04:57,074: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-29 20:04:57,075: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-29 20:04:57,079: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-29 20:04:57,081: INFO: common: created directory at: artifacts]
[2023-09-29 20:04:57,082: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-29 20:04:57,087: INFO: 2429056447: File already exists of size: ~ 23 KB]


### Now it's the turn to convert this notebook into modular coding :)